Create a model using built-in library from Pytorch.
This code closelt follows nn_tutorial notebook.

In [9]:
import csv, torch, math, os, pickle
from torch import nn
from torch import optim

global chroma_shape, epochs, train_bs, validate_bs, lr

In [2]:
## read attr and tar in .pkl files
## Return: (1) a list of padded attr arrays
##         (2) a list of paddrd tar arrays
def read_data():
    att_file = open(r'temp_chroma_attr.pkl', 'rb')
    x_train = pickle.load(att_file)
    
    att_file.close()
    
    tar_file = open(r'temp_chroma_tar.pkl', 'rb')
    y_train = pickle.load(tar_file)
    
    tar_file.close()
    return x_train, y_train

x_train, y_train = read_data()


In [12]:
## Get the shape of a padded instance for model construction
chroma_shape = x_train[0]

lr = 0.5
epochs = 10
train_bs = 5
validate_bs = train_bs*2
train_n = len(x_train)

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim = 1)
    return (preds == yb).float().mean()

Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [ ]:
train_n, c = x_train.shape
valid_n, c = x_valid.shape
D_out = 3
lr = 0.5
epochs = 10
train_bs = 5
validate_bs = train_bs*2

import torch.nn.functional as F

loss_func = F.cross_entropy


In [10]:
class SoundRecognition(nn.Module):
    def __init__(self):
        super().__init__()
        
        ## This one line does the same thing as the two lines below 
        ## and [xb @ self.weights + self.bias]
        self.lin = nn.Linear(c, D_out)
#         self.weights = nn.Parameter(torch.randn(c, D_out) / math.sqrt(c))
#         self.bias = nn.Parameter(torch.zeros(D_out))
        
    def forward(self, xb):
        return self.lin(xb)
    
class SoundRecognition_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(16, 3, kernel_size=3, stride=2)

    def forward(self, xb):
        xb = xb.view(-1, 1, chroma_shape[0], chroma_shape[1])
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

## Get the model and optim object that will be used to update model parameters
def get_model():
    model = SoundRecognition_CNN()
    return model, optim.SGD(model.parameters(), lr = lr)

In [13]:
model, opt = get_model()

def fit():
    for epoch in range(epochs):
        
        print('Training')
        model.train()
        for i in range((train_n - 1) // train_bs + 1):
            
            start_i = i * train_bs
            end_i = start_i + train_bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()
        
            #print('Acc: ', accuracy(pred, yb))
        
        ####validate at each epoch
        print('Validating')
        model.eval()
        loss = []
        acc = []
        with torch.no_grad():
#             for i in range((valid_n - 1) // validate_bs + 1):
#                 start_i = i * validate_bs
#                 end_i = start_i + validate_bs
#                 xb = x_valid[start_i:end_i]
#                 yb = y_valid[start_i:end_i]
#                 pred = model(xb)
#                 #print('----', yb)
#                 loss.append(loss_func(pred, yb))
#                 acc.append(accuracy(pred, yb))

            for i in range((train_n - 1) // train_bs + 1):

                start_i = i * train_bs
                end_i = start_i + train_bs
                xb = x_train[start_i:end_i]
                yb = y_train[start_i:end_i]
                pred = model(xb)
                loss.append(loss_func(pred, yb))
                acc.append(accuracy(pred, yb))
        
        #print(loss)
        valid_loss = sum(loss)
        valid_acc = sum(acc)
        
    
        print('Epoch: ', epoch, ' | Loss: ', valid_loss, ' | Accuracy: ', valid_acc/len(y_train))
    

fit()


Training


AttributeError: 'list' object has no attribute 'view'